# Script 1. Preparacion de los datos


## Parte 0. Mount

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Parte 1. Datos crudos - Union de todos los datasets

**Output: ** df_final, incluye los datos de 2020-2024, las zonas en formato encoded, fecha y hora separadas
Importante, la información no ha sido modificada, es decir, NaNs, Nulls, y datos atípicos siguen dentro del dataset

### Procesamiento Dataset 2023-2024

In [ ]:
import pandas as pd

# Ruta del archivo
archivo_principal = '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/TODO/DATOS_HISTÓRICOS_2023_2024_TODAS_ESTACIONES_ITESM.csv'

# Nombres de las zonas y sus respectivos archivos
zonas = ['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'NORESTE2', 'SURESTE2', 'SUROESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE3', 'NOROESTE3']

# Columnas que tiene cada conjunto
columnas_por_zona = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'PRS', 'RAINF', 'RH', 'SO2', 'SR', 'TOUT', 'WSR', 'WDR']

# Lee el archivo principal (usando low_memory=False y dtype=str para evitar advertencias)
df = pd.read_csv(archivo_principal, header=0, index_col=False, low_memory=False, dtype=str)

# Ubicación donde se guardarán los archivos
ruta_guardado = '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2023_2024'

# Procesa cada conjunto de columnas
for i, zona in enumerate(zonas):
    # Determina las columnas para esta zona (se asume que hay una columna vacía entre cada conjunto de datos)
    inicio = i * (len(columnas_por_zona) + 1) + 1  # Se incluye la columna vacía como separación
    fin = inicio + len(columnas_por_zona)

    # Extrae la columna "date" y el conjunto de columnas para la zona
    columnas_zona = ['date'] + list(df.columns[inicio:fin])

    # Filtra el DataFrame por esas columnas
    df_zona = df[columnas_zona]

    # Renombra las columnas para que no incluyan los sufijos numéricos
    df_zona.columns = ['date'] + columnas_por_zona

    # Guarda el archivo CSV para cada zona
    nombre_archivo = f"{zona}_2023_2024.csv"
    df_zona.to_csv(ruta_guardado + nombre_archivo, index=False)

print("Listo :^)")


Listo :^)


### Union bases de datos 2020-2024

In [ ]:
import os
import pandas as pd

# Rutas de las carpetas con archivos CSV
#folders = [
#    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2020_2021/',
#    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2022_2023/',
#    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2023_2024/'
#]

folders = [
    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2022_2023/',
    '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/2023_2024/'
]

# Ruta de destino para el archivo combinado
output_folder = '/content/gdrive/MyDrive/SIMA/FINAL/'
output_file = os.path.join(output_folder, 'combined_df_2022_2024.csv')

# Crear una lista para almacenar los DataFrames
df_list = []

# Leer y concatenar los archivos CSV de cada carpeta
for folder in folders:
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder, filename)
            # Leer el archivo CSV
            df = pd.read_csv(file_path)

            # Extraer la zona del nombre del archivo
            zona = filename.split('_')[0]  # Obtiene el primer segmento del nombre (por ejemplo, 'CENTRO')

            # Agregar la nueva columna de zona al DataFrame
            df['zona'] = zona

            # Agregar el DataFrame a la lista
            df_list.append(df)

# Concatenar todos los DataFrames
combined_df = pd.concat(df_list, ignore_index=True)

# Guardar el archivo combinado en la nueva ubicación
combined_df.to_csv(output_file, index=False)

print(f'Archivos combinados y guardados en: {output_file}')

# OUTPUT EN /content/gdrive/MyDrive/SIMA/FINAL/BASES_2020_2024_SP.csv
# NOMBRE df: combined_df

print("Listo :^)")


Archivos combinados y guardados en: /content/gdrive/MyDrive/SIMA/FINAL/combined_df_2022_2024.csv
Listo :^)


In [ ]:
combined_df_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/combined_df_2022_2024.csv')

<ipython-input-4-7f9a3463cd53>:1: DtypeWarning: Columns (5,6,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/combined_df_2022_2024.csv')


### Añadir columna de Zona

In [ ]:
# Cargar los DataFrames
combined_df_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/combined_df_2022_2024.csv')
df_UBI = pd.read_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/UBICACIONES/UBI.csv')

<ipython-input-5-72c41a448948>:2: DtypeWarning: Columns (5,6,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/combined_df_2022_2024.csv')


In [ ]:
import pandas as pd
# Lista de columnas donde se debe reemplazar la coma por punto
columnas_a_reemplazar = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'PRS', 'RAINF', 'RH', 'SO2', 'SR', 'TOUT', 'WSR', 'WDR']

# Reemplazar comas por puntos en las columnas seleccionadas
combined_df_2022_2024[columnas_a_reemplazar] = combined_df_2022_2024[columnas_a_reemplazar].replace({',': '.'}, regex=True)

# Convertir las columnas a tipo numérico
combined_df_2022_2024[columnas_a_reemplazar] = combined_df_2022_2024[columnas_a_reemplazar].apply(pd.to_numeric, errors='coerce')

# Verificar los cambios
print(combined_df_2022_2024[columnas_a_reemplazar].head())

     CO   NO   NO2   NOX    O3   PM10   PM2.5    PRS  RAINF    RH  SO2   SR  \
0  2.60  3.5  44.9  48.5  15.0  134.0   91.00  705.5    0.0  46.0  5.3  0.0   
1  2.23  2.9  32.9  36.0  19.0  141.0  112.61  705.2    0.0  48.0  6.1  0.0   
2  1.99  2.9  27.6  30.7  21.0  117.0   92.46  705.1    0.0  46.0  5.8  0.0   
3  2.03  NaN   NaN   NaN  18.0  108.0   69.20  704.8    0.0  46.0  6.0  0.0   
4  1.86  NaN   NaN   NaN  20.0  106.0   68.03  704.8    0.0  46.0  5.6  0.0   

    TOUT  WSR    WDR  
0  21.37  3.4  267.0  
1  20.83  3.3  259.0  
2  20.52  5.9  233.0  
3  20.12  5.1  220.0  
4  19.85  4.7  244.0  


In [ ]:

# Función para dividir la columna 'date' en 'date' y 'time'
def split_date_time(df):
    # Convertir la columna 'date' a tipo datetime
    df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y %H:%M')

    # Extraer la fecha y la hora en columnas separadas
    df['time'] = df['date'].dt.time   # Extraer la hora
    df['date'] = df['date'].dt.date   # Extraer la fecha

    return df

# Aplicar la función a cada DataFrame

combined_split_df_2022_2024 = split_date_time(combined_df_2022_2024)

# Guardar los DataFrames con las nuevas columnas


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Función para codificar la columna 'Zona' y generar el diccionario de encoding
def encode_zona(df):
    # Crear el codificador
    le = LabelEncoder()

    # Ajustar el codificador a la columna 'Zona' y transformar los valores
    df['zona_encoded'] = le.fit_transform(df['zona'])

    # Crear el diccionario de encoding
    zona_dict = dict(zip(le.classes_, le.transform(le.classes_)))

    # Eliminar la columna 'Zona' original
    df = df.drop(columns=['zona'])

    return df, zona_dict

# Aplicar la codificación a cada DataFrame
combined_split_df_2022_2024, zona_dict_2022_2024 = encode_zona(combined_split_df_2022_2024)

# Mostrar los diccionarios de encoding para cada DataFrame
print("Diccionario de encoding para 2022-2024:", zona_dict_2022_2024)

# Guardar los DataFrames con la columna 'Zona' eliminada y la columna 'Zona_encoded'
combined_split_df_2022_2024.to_csv('/content/gdrive/MyDrive/SIMA/FINAL/base_cruda_2022_2024.csv', index=False)

print("Los archivos con la columna 'Zona' eliminada han sido guardados.")

# OUTPUT '/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/TODO/BASES_2020_2024_SP_encoded.csv

print("Listo :^)")


Diccionario de encoding para 2022-2024: {'CENTRO': 0, 'NORESTE': 1, 'NORESTE2': 2, 'NORESTE3': 3, 'NOROESTE': 4, 'NOROESTE2': 5, 'NOROESTE3': 6, 'NORTE': 7, 'NORTE2': 8, 'SUR': 9, 'SURESTE': 10, 'SURESTE2': 11, 'SURESTE3': 12, 'SUROESTE': 13, 'SUROESTE2': 14}
Los archivos con la columna 'Zona' eliminada han sido guardados.
Listo :^)


### OUTPUT

In [ ]:
base_cruda_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/base_cruda_2022_2024.csv')

## Parte 2. Limpieza Informacion

In [ ]:
# Definir columnas relevantes para la imputación
columnas_relevantes = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'PRS', 'RAINF', 'RH', 'SO2', 'SR', 'TOUT', 'WSR', 'WDR']

# Convertir solo las columnas relevantes a formato numérico
base_cruda_2022_2024[columnas_relevantes] = base_cruda_2022_2024[columnas_relevantes].apply(pd.to_numeric, errors='coerce')

### Eliminacion de outliers, NaNs y Nulls

In [ ]:
import pandas as pd

# Función para eliminar los outliers usando el rango intercuartílico (IQR)
def eliminar_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Contar outliers
    outliers_count = ((col < lower_bound) | (col > upper_bound)).sum()

    # Reemplazar outliers por NaN
    col_cleaned = col.where((col >= lower_bound) & (col <= upper_bound))

    return col_cleaned, outliers_count

# Inicializar un DataFrame para almacenar la cantidad de outliers
outliers_info = {}

# Aplicar la función para eliminar outliers en las columnas relevantes
for col in columnas_relevantes:
    base_cruda_2022_2024[col], count = eliminar_outliers(base_cruda_2022_2024[col])
    outliers_info[col] = count

base_cocinada_2022_2024 = base_cruda_2022_2024

# Mostrar resultados, asegurando que se mantengan las columnas 'date', 'time', y 'Zona_encoded'
print("Dataframe con outliers reemplazados por NaN:")
print(base_cocinada_2022_2024)

# Mostrar la cantidad de outliers por columna
print("\nCantidad de outliers reemplazados por columna:")
for column, count in outliers_info.items():
    print(f"{column}: {count} outliers")

print("Listo :^)")


Dataframe con outliers reemplazados por NaN:
              date    CO   NO   NO2   NOX    O3   PM10  PM2.5    PRS  RAINF  \
0       2022-01-01  2.60  3.5   NaN  48.5  15.0  134.0    NaN  705.5    0.0   
1       2022-01-01  2.23  2.9  32.9  36.0  19.0    NaN    NaN  705.2    0.0   
2       2022-01-01  1.99  2.9  27.6  30.7  21.0  117.0    NaN  705.1    0.0   
3       2022-01-01  2.03  NaN   NaN   NaN  18.0  108.0    NaN  704.8    0.0   
4       2022-01-01  1.86  NaN   NaN   NaN  20.0  106.0    NaN  704.8    0.0   
...            ...   ...  ...   ...   ...   ...    ...    ...    ...    ...   
413850  2024-07-31  1.18  4.2  11.1  15.2  16.0  117.0  13.73  707.5    0.0   
413851  2024-07-31  1.33  4.4  12.5  16.8  13.0   90.0  13.05  707.9    0.0   
413852  2024-07-31  1.47  4.6  12.1  16.6  12.0   90.0  12.72  708.4    0.0   
413853  2024-07-31  1.26  4.8  11.3  16.0  12.0  104.0  13.63  709.1    0.0   
413854  2024-07-31   NaN  NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   

      

### OUTPUT

In [ ]:
base_cocinada_2022_2024.to_csv('/content/gdrive/MyDrive/SIMA/FINAL/base_cocinada_2022_2024.csv', index=False)
base_cocinada_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/base_cocinada_2022_2024.csv')

## Parte 3. Imputacion

In [ ]:
!pip install fancyimpute


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.0 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=106a93995ff11c6f148b5ec77c6519eeab3fc8ab7c502bd2280871e6fb2cbd4a
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=5287716ed4c229d597ec84d1a710cbf492e18262ec0bf00ef5a525b9612916b7
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


### Uso del algoritmo MICE

### Inclusion de diccionario de zonas

In [ ]:
import pandas as pd
# Diccionario de encoding de las zonas
zona_dict = {
    'CENTRO': 0,
    'NORESTE': 1,
    'NORESTE2': 2,
    'NORESTE3': 3,
    'NOROESTE': 4,
    'NOROESTE2': 5,
    'NOROESTE3': 6,
    'NORTE': 7,
    'NORTE2': 8,
    'SUR': 9,
    'SURESTE': 10,
    'SURESTE2': 11,
    'SURESTE3': 12,
    'SUROESTE': 13,
    'SUROESTE2': 14,
}

import pandas as pd

# Dataframes: df_UBI y base_cocinada_2020_2024
# Supongo que df_UBI tiene la columna 'Zona' que coincide con las claves del diccionario zona_dict

# Invertimos el diccionario zona_dict para mapear los valores codificados a las zonas
inverse_zona_dict = {v: k for k, v in zona_dict.items()}

# Mapeamos la columna 'zona_encoded' en base_cocinada_2020_2024 para obtener los nombres de las zonas
base_cocinada_2022_2024['Zona'] = base_cocinada_2022_2024['zona_encoded'].map(inverse_zona_dict)

# Unimos los dataframes en base a la columna 'Zona'
df_merged = pd.merge(base_cocinada_2022_2024, df_UBI[['Zona', 'location']], on='Zona', how='left')

# Dividimos la columna 'location' en latitud y longitud
df_merged[['latitud', 'longitud']] = df_merged['location'].str.split(',', expand=True)

# Convertimos las columnas 'latitud' y 'longitud' a tipo float
df_merged['latitud'] = df_merged['latitud'].astype(float)
df_merged['longitud'] = df_merged['longitud'].astype(float)

# Eliminamos la columna 'location'
df_merged = df_merged.drop(columns=['location', 'Zona', 'RAINF'])

print("Listo :^)")



Listo :^)


In [ ]:
# Filtrar filas donde la latitud o longitud son nulas
#zonas_con_nans = df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024[(df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['latitud'].isnull()) |
                                                                             #0# (df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['longitud'].isnull())]

# Mostrar las zonas que tienen nulos en latitud o longitud
#zonas_faltantes = zonas_con_nans['zona_encoded'].unique()
#print(f"Zonas con coordenadas faltantes: {zonas_faltantes}")


In [ ]:
import pandas as pd

# Cargar el archivo CSV
base_cocinada_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/base_cocinada_2022_2024.csv')

# Verificar si hay valores faltantes
missing_values = base_cocinada_2022_2024.isna().sum()
print("Valores faltantes por columna:")
print(missing_values[missing_values > 0])

# Verificar tipos de datos por columna
print("\nTipos de datos:")
print(base_cocinada_2022_2024.dtypes)

# Verificar si hay filas con longitud diferente al número de columnas
expected_columns = len(base_cocinada_2022_2024.columns)
for i, row in base_cocinada_2022_2024.iterrows():
    if len(row) != expected_columns:
        print(f"\nFila {i} tiene una cantidad inesperada de columnas.")

# Verificar si hay filas duplicadas
duplicated_rows = base_cocinada_2022_2024.duplicated().sum()
if duplicated_rows > 0:
    print(f"\nNúmero de filas duplicadas: {duplicated_rows}")
else:
    print("\nNo se encontraron filas duplicadas.")


Valores faltantes por columna:
CO        26895
NO        60987
NO2       29521
NOX       47810
O3        28895
PM10      32876
PM2.5    114591
PRS       12357
RAINF     11096
RH        34019
SO2       49483
SR        36801
TOUT      10325
WSR       17022
WDR       37753
dtype: int64

Tipos de datos:
date             object
CO              float64
NO              float64
NO2             float64
NOX             float64
O3              float64
PM10            float64
PM2.5           float64
PRS             float64
RAINF           float64
RH              float64
SO2             float64
SR              float64
TOUT            float64
WSR             float64
WDR             float64
time             object
zona_encoded      int64
dtype: object

Número de filas duplicadas: 65814


In [ ]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from fancyimpute import IterativeImputer

# Suponiendo que ya tienes cargado el dataframe df_merged
# Convertimos 'date' a tipo datetime para extraer el mes
df_merged['date'] = pd.to_datetime(df_merged['date'])

# Lista de columnas a imputar
columns_to_impute = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2', 'WSR', 'WDR']

# Creamos una copia para no modificar las columnas que no necesitan imputación
df_imputed = df_merged.copy()

# Imputación por cada mes utilizando MICE
for month in df_merged['date'].dt.month.unique():
    # Filtramos los datos correspondientes al mes actual
    df_month = df_merged[df_merged['date'].dt.month == month]

    # Realizamos la imputación utilizando MICE con regresión bayesiana
    imputer = IterativeImputer(estimator=BayesianRidge(), random_state=0, max_iter=35, tol=1e-3)

    # Imputamos solo las columnas especificadas, dejando las demás sin cambios
    imputed_data = imputer.fit_transform(df_month[columns_to_impute])

    # Reemplazamos los valores imputados en el dataframe original
    df_imputed.loc[df_imputed['date'].dt.month == month, columns_to_impute] = imputed_data

# Ahora df_imputed tiene los valores imputados, manteniendo las demás columnas sin cambios


In [ ]:
df_imputed

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RH,SO2,SR,TOUT,WSR,WDR,time,zona_encoded,latitud,longitud
0,2022-01-01,2.600000,3.500000,44.635472,48.500000,15.000000,134.000000,46.000132,705.5,46.0,5.300000,0.000,21.37,3.400000,267.000000,00:00:00,0,25.676020,-100.335847
1,2022-01-01,2.230000,2.900000,32.900000,36.000000,19.000000,90.487099,32.866904,705.2,48.0,6.100000,0.000,20.83,3.300000,259.000000,01:00:00,0,25.676020,-100.335847
2,2022-01-01,1.990000,2.900000,27.600000,30.700000,21.000000,117.000000,35.302492,705.1,46.0,5.800000,0.000,20.52,5.900000,233.000000,02:00:00,0,25.676020,-100.335847
3,2022-01-01,2.030000,10.988332,26.355984,37.435607,18.000000,108.000000,33.597876,704.8,46.0,6.000000,0.000,20.12,5.100000,220.000000,03:00:00,0,25.676020,-100.335847
4,2022-01-01,1.860000,10.671060,25.491311,36.196048,20.000000,106.000000,31.740669,704.8,46.0,5.600000,0.000,19.85,4.700000,244.000000,04:00:00,0,25.676020,-100.335847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413850,2024-07-31,1.180000,4.200000,11.100000,15.200000,16.000000,117.000000,13.730000,707.5,46.0,2.800000,0.003,29.87,13.100000,63.000000,19:00:00,6,25.785307,-100.463958
413851,2024-07-31,1.330000,4.400000,12.500000,16.800000,13.000000,90.000000,13.050000,707.9,48.0,2.700000,0.000,28.93,11.800000,41.000000,20:00:00,6,25.785307,-100.463958
413852,2024-07-31,1.470000,4.600000,12.100000,16.600000,12.000000,90.000000,12.720000,708.4,50.0,3.000000,0.000,28.31,11.600000,41.000000,21:00:00,6,25.785307,-100.463958
413853,2024-07-31,1.260000,4.800000,11.300000,16.000000,12.000000,104.000000,13.630000,709.1,58.0,3.000000,0.000,27.25,15.300000,35.000000,22:00:00,6,25.785307,-100.463958


In [ ]:
# Verificar que ya no haya NaNs
df_imputed.isna().sum()

,0
date,0
CO,0
NO,0
NO2,0
NOX,0
O3,0
PM10,0
PM2.5,0
PRS,12357
RH,34019


In [ ]:
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024 = df_imputed

### OUTPUT

In [ ]:
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.to_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.csv', index=False)

df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.csv')

## Parte 4. Estandarizacion

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Supongamos que df_sin_outliers ya está definido y contiene las columnas de interés

# Seleccionar las columnas a estandarizar
columns_to_standardize = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2']

# Estandarización (Media 0, Desviación Estándar 1)
scaler = StandardScaler()
df_completo_estandarizado_2022_2024 = df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.copy()  # Hacer una copia para no modificar el original
df_completo_estandarizado_2022_2024[columns_to_standardize] = scaler.fit_transform(df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024[columns_to_standardize])

print("Listo :^)")


Listo :^)


### OUTPUT

In [ ]:
df_completo_estandarizado_2022_2024.to_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_completo_estandarizado_2022_2024.csv', index=False)
df_completo_estandarizado_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_completo_estandarizado_2022_2024.csv')

## Parte 5. Resultados

In [ ]:
### 1. Dataframe y archivo para analisis descriptivo crudo
base_cruda_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/base_cruda_2022_2024.csv')

### 2. Dataframe y archivo para analisis descriptivo procesado, desarrollo supuestos estadisticos (Exploracion de datos nulos y outliers)
base_cocinada_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/base_cocinada_2022_2024.csv')

### 3. Dataframe y archivo para analisis descriptivo procesado, desarrollo supuestos estadisticos (Histogramas y estadistica general)
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.csv')

### 4. Dataframe y archivo para analisis descriptivo procesado + estandarizacion, desarrollo supuestos estadisticos (Histogramas y estadistica general)
df_completo_estandarizado_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_completo_estandarizado_2022_2024.csv')

### 5. Dataframe y archivo para clusterización
df_completo_estandarizado_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_completo_estandarizado_2022_2024.csv')

### 6. Dataframe y archivo para interpolacion y mapa
df_completo_estandarizado_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_completo_estandarizado_2022_2024.csv')

## 7. Ubicaciones y datos de estaciones
df_UBI = pd.read_csv('/content/gdrive/MyDrive/SIMA/BASES_DE_DATOS/PROCESADO/UBICACIONES/UBI.csv')


In [ ]:
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RH,SO2,SR,TOUT,WSR,WDR,time,zona_encoded,latitud,longitud
0,2022-01-01,2.600000,3.500000,44.635472,48.500000,15.000000,134.000000,46.000132,705.5,46.0,5.300000,0.000,21.37,3.400000,267.000000,00:00:00,0,25.676020,-100.335847
1,2022-01-01,2.230000,2.900000,32.900000,36.000000,19.000000,90.487099,32.866904,705.2,48.0,6.100000,0.000,20.83,3.300000,259.000000,01:00:00,0,25.676020,-100.335847
2,2022-01-01,1.990000,2.900000,27.600000,30.700000,21.000000,117.000000,35.302492,705.1,46.0,5.800000,0.000,20.52,5.900000,233.000000,02:00:00,0,25.676020,-100.335847
3,2022-01-01,2.030000,10.988332,26.355984,37.435607,18.000000,108.000000,33.597876,704.8,46.0,6.000000,0.000,20.12,5.100000,220.000000,03:00:00,0,25.676020,-100.335847
4,2022-01-01,1.860000,10.671060,25.491311,36.196048,20.000000,106.000000,31.740669,704.8,46.0,5.600000,0.000,19.85,4.700000,244.000000,04:00:00,0,25.676020,-100.335847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413850,2024-07-31,1.180000,4.200000,11.100000,15.200000,16.000000,117.000000,13.730000,707.5,46.0,2.800000,0.003,29.87,13.100000,63.000000,19:00:00,6,25.785307,-100.463958
413851,2024-07-31,1.330000,4.400000,12.500000,16.800000,13.000000,90.000000,13.050000,707.9,48.0,2.700000,0.000,28.93,11.800000,41.000000,20:00:00,6,25.785307,-100.463958
413852,2024-07-31,1.470000,4.600000,12.100000,16.600000,12.000000,90.000000,12.720000,708.4,50.0,3.000000,0.000,28.31,11.600000,41.000000,21:00:00,6,25.785307,-100.463958
413853,2024-07-31,1.260000,4.800000,11.300000,16.000000,12.000000,104.000000,13.630000,709.1,58.0,3.000000,0.000,27.25,15.300000,35.000000,22:00:00,6,25.785307,-100.463958


In [ ]:
!pip install fancyimpute

In [ ]:
import pandas as pd

# Leer el DataFrame desde el archivo CSV
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.csv')

# Verificar si la columna 'date' existe en el DataFrame
if 'date' not in df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.columns:
    raise ValueError("La columna 'date' no existe en el DataFrame.")

# Convertir la columna de fecha a tipo datetime
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'] = pd.to_datetime(df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'], errors='coerce')

# Comprobar si hay fechas no válidas (NaT) tras la conversión
if df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'].isnull().any():
    print("Algunas fechas no son válidas y se han convertido a NaT. Por favor, verifica los datos de entrada.")

# Solicitar al usuario que ingrese la fecha hasta la cual quiere conservar el DataFrame
fecha_usuario = input("Introduce la fecha (en formato YYYY-MM-DD): ")

# Convertir la fecha ingresada por el usuario a tipo datetime, manejando errores
try:
    fecha_usuario = pd.to_datetime(fecha_usuario)
except ValueError:
    raise ValueError("La fecha ingresada no es válida. Asegúrate de usar el formato YYYY-MM-DD.")

# Crear un nuevo DataFrame eliminando las filas posteriores a la fecha especificada
nuevo_dataframe = df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024[df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'] <= fecha_usuario]

# Mostrar las primeras filas del nuevo DataFrame
print(nuevo_dataframe.tail())

# Si deseas guardar el nuevo DataFrame en un archivo CSV, puedes descomentar la siguiente línea:
# nuevo_dataframe.to_csv('/content/gdrive/MyDrive/SIMA/FINAL/nuevo_dataframe.csv', index=False)


Introduce la fecha (en formato YYYY-MM-DD): 2024-02-02
             date    CO   NO   NO2   NOX    O3  PM10      PM2.5    PRS    RH  \
409530 2024-02-02  1.61  2.6  15.6  18.1  58.0  99.0  25.330218  699.1  26.0   
409531 2024-02-02  1.49  2.5  21.8  24.1  41.0  81.0  16.366897  699.1  30.0   
409532 2024-02-02  1.48  3.2  27.5  30.6  22.0  79.0  16.500544  699.3  34.0   
409533 2024-02-02  1.39  2.4  14.6  16.9  28.0  83.0  17.744261  699.7  39.0   
409534 2024-02-02  1.39  2.4  12.5  14.7  28.0  53.0  11.917027  700.2  42.0   

        SO2     SR   TOUT   WSR   WDR      time  zona_encoded    latitud  \
409530  7.8  0.177  29.22   7.6  46.0  19:00:00             6  25.785307   
409531  5.0  0.010  27.71  12.0  11.0  20:00:00             6  25.785307   
409532  4.6  0.000  26.76  14.3   9.0  21:00:00             6  25.785307   
409533  5.1  0.000  25.49  12.5  15.0  22:00:00             6  25.785307   
409534  5.5  0.000  24.66  10.0  40.0  23:00:00             6  25.785307   

       

In [ ]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from fancyimpute import IterativeImputer

# Asegúrate de que ya tienes cargado nuevo_dataframe y es una copia del DataFrame original
nuevo_dataframe = nuevo_dataframe.copy()  # Hacemos una copia

# Convertimos 'date' a tipo datetime
nuevo_dataframe['date'] = pd.to_datetime(nuevo_dataframe['date'])

# Obtiene la última fecha del DataFrame
ultima_fecha = nuevo_dataframe['date'].max()

# Generamos nuevas filas para cada hora del día siguiente
nuevas_filas = []
for hour in range(24):
    # Creamos un nuevo datetime para cada hora del día
    nueva_fecha = ultima_fecha + pd.DateOffset(days=1, hours=hour)
    # Usamos NaN para las columnas a imputar, asegurándonos de que coincida con el número de columnas
    nuevas_filas.append([nueva_fecha] + [None] * (len(nuevo_dataframe.columns) - 1))

# Creamos un nuevo DataFrame con las filas nuevas
nuevas_filas_df = pd.DataFrame(nuevas_filas, columns=nuevo_dataframe.columns)

# Agregamos las nuevas filas al DataFrame original
nuevo_dataframe = pd.concat([nuevo_dataframe, nuevas_filas_df], ignore_index=True)

# Lista de columnas a imputar
columns_to_impute = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2', 'WSR', 'WDR']

# Creamos una copia para no modificar las columnas que no necesitan imputación
df_imputed = nuevo_dataframe.copy()

# Imputación por cada mes utilizando MICE
for month in df_imputed['date'].dt.month.unique():
    # Filtramos los datos correspondientes al mes actual
    df_month = df_imputed[df_imputed['date'].dt.month == month]

    # Asegúrate de que hay al menos un valor no nulo para la imputación
    if df_month[columns_to_impute].notnull().any().any():  # Verificamos si hay valores no nulos
        # Realizamos la imputación utilizando MICE con regresión bayesiana
        imputer = IterativeImputer(estimator=BayesianRidge(), random_state=0, max_iter=35, tol=1e-3)

        # Imputamos solo las columnas especificadas, dejando las demás sin cambios
        imputed_data = imputer.fit_transform(df_month[columns_to_impute])

        # Reemplazamos los valores imputados en el dataframe original
        df_imputed.loc[df_imputed['date'].dt.month == month, columns_to_impute] = imputed_data

# Ahora df_imputed tiene los valores imputados, manteniendo las demás columnas sin cambios
print(df_imputed.tail())  # Muestra las primeras filas del DataFrame imputado


<ipython-input-28-30ee5ae50544>:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  nuevo_dataframe = pd.concat([nuevo_dataframe, nuevas_filas_df], ignore_index=True)


                      date        CO        NO        NO2        NOX  \
349074 2024-02-03 19:00:00  1.464085  8.659681  18.323866  26.914487   
349075 2024-02-03 20:00:00  1.464085  8.659681  18.323866  26.914487   
349076 2024-02-03 21:00:00  1.464085  8.659681  18.323866  26.914487   
349077 2024-02-03 22:00:00  1.464085  8.659681  18.323866  26.914487   
349078 2024-02-03 23:00:00  1.464085  8.659681  18.323866  26.914487   

               O3      PM10      PM2.5  PRS  RH       SO2  SR  TOUT       WSR  \
349074  22.645937  60.85991  18.168653  NaN NaN  4.184621 NaN   NaN  7.826908   
349075  22.645937  60.85991  18.168653  NaN NaN  4.184621 NaN   NaN  7.826908   
349076  22.645937  60.85991  18.168653  NaN NaN  4.184621 NaN   NaN  7.826908   
349077  22.645937  60.85991  18.168653  NaN NaN  4.184621 NaN   NaN  7.826908   
349078  22.645937  60.85991  18.168653  NaN NaN  4.184621 NaN   NaN  7.826908   

               WDR  time zona_encoded  latitud  longitud  
349074  126.664175  N

In [ ]:
import pandas as pd

# Leer el DataFrame desde el archivo CSV
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024 = pd.read_csv('/content/gdrive/MyDrive/SIMA/FINAL/df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.csv')

# Verificar si la columna 'date' existe en el DataFrame
if 'date' not in df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024.columns:
    raise ValueError("La columna 'date' no existe en el DataFrame.")

# Convertir la columna de fecha a tipo datetime
df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'] = pd.to_datetime(df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'], errors='coerce')

# Comprobar si hay fechas no válidas (NaT) tras la conversión
if df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'].isnull().any():
    print("Algunas fechas no son válidas y se han convertido a NaT. Por favor, verifica los datos de entrada.")

# Definir la fecha hasta la cual se desean mostrar los datos
fecha_filtro = '2024-02-03'

# Convertir la fecha a tipo datetime
fecha_filtro = pd.to_datetime(fecha_filtro)

# Filtrar el DataFrame hasta la fecha especificada
datos_filtrados = df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024[df_Completo_Imputado_Todascolumnas_SinEstandarizar_2022_2024['date'] <= fecha_filtro]

# Mostrar las primeras filas del DataFrame filtrado
print(datos_filtrados.tail())

# Si deseas ver todos los datos filtrados, puedes descomentar la siguiente línea:
# print(datos_filtrados)


             date    CO    NO   NO2   NOX    O3   PM10      PM2.5    PRS  \
409554 2024-02-03  1.26   3.5   7.9  11.3  43.0   95.0  19.524931  698.6   
409555 2024-02-03  1.32   3.4  14.2  17.5  34.0  113.0  19.408105  698.8   
409556 2024-02-03  1.38   4.2  17.5  21.6  27.0   37.0   8.618883  699.3   
409557 2024-02-03  1.61  10.9  26.8  37.6  15.0   35.0  13.249652  700.1   
409558 2024-02-03  1.77   9.7  26.8  36.5  12.0   47.0  18.952035  700.5   

          RH  SO2     SR   TOUT        WSR    WDR      time  zona_encoded  \
409554   5.0  2.9  0.169  27.53  11.320501  255.0  19:00:00             6   
409555   6.0  3.0  0.009  26.28  19.600000  254.0  20:00:00             6   
409556   7.0  3.1  0.000  24.53  13.100000  232.0  21:00:00             6   
409557  11.0  3.6  0.000  22.77   8.300000  265.0  22:00:00             6   
409558  13.0  3.2  0.000  21.49   3.500000  278.0  23:00:00             6   

          latitud    longitud  
409554  25.785307 -100.463958  
409555  25.78530

In [ ]:
df_imputed.tail()


,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RH,SO2,SR,TOUT,WSR,WDR,time,zona_encoded,latitud,longitud
349074,2024-02-03 19:00:00,1.464085,8.659681,18.323866,26.914487,22.645937,60.85991,18.168653,NaN,NaN,4.184621,NaN,NaN,7.826908,126.664175,None,None,NaN,NaN
349075,2024-02-03 20:00:00,1.464085,8.659681,18.323866,26.914487,22.645937,60.85991,18.168653,NaN,NaN,4.184621,NaN,NaN,7.826908,126.664175,None,None,NaN,NaN
349076,2024-02-03 21:00:00,1.464085,8.659681,18.323866,26.914487,22.645937,60.85991,18.168653,NaN,NaN,4.184621,NaN,NaN,7.826908,126.664175,None,None,NaN,NaN
349077,2024-02-03 22:00:00,1.464085,8.659681,18.323866,26.914487,22.645937,60.85991,18.168653,NaN,NaN,4.184621,NaN,NaN,7.826908,126.664175,None,None,NaN,NaN
349078,2024-02-03 23:00:00,1.464085,8.659681,18.323866,26.914487,22.645937,60.85991,18.168653,NaN,NaN,4.184621,NaN,NaN,7.826908,126.664175,None,None,NaN,NaN


In [ ]:
import pandas as pd

# Suponemos que 'df_imputed' y 'datos_filtrados' ya están disponibles y tienen la columna 'date'
# Seleccionamos una fecha específica para la comparación (usando la última fecha común en ambos datasets)
fecha_comparacion = '2024-02-03'  # Cambiar a la fecha que desees usar para la comparación

# Filtramos los datos para la fecha específica en ambos datasets
valores_imputados = df_imputed[df_imputed['date'] == fecha_comparacion].set_index('date')
valores_reales = datos_filtrados[datos_filtrados['date'] == fecha_comparacion].set_index('date')

# Aseguramos que ambas tablas están alineadas y tienen las mismas columnas para comparar
columnas_comunes = ['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'SO2', 'WSR', 'WDR']
valores_imputados = valores_imputados[columnas_comunes]
valores_reales = valores_reales[columnas_comunes]

# Calculamos la diferencia porcentual ((valor_imputado - valor_real) / valor_real) * 100
diferencias_porcentuales = ((valores_imputados - valores_reales) / valores_reales) * 100

# Mostramos las diferencias porcentuales
print(diferencias_porcentuales)


                   CO          NO         NO2         NOX          O3  \
date                                                                    
2024-02-03 -41.669926  -25.014841  -45.301892  -40.285110  352.918747   
2024-02-03 -39.749594  -22.189248  -40.118084  -35.544489  352.918747   
2024-02-03 -32.840144  -64.293863  -32.879611  -47.535113  277.432289   
2024-02-03 -29.948092  -58.366918  -30.591416  -42.856715  223.513391   
2024-02-03 -28.581226  -58.958858  -33.124576  -44.391555  277.432289   
...               ...         ...         ...         ...         ...   
2024-02-03  16.197212  147.419458  131.947675  138.181303  -47.335029   
2024-02-03  10.915520  154.696501   29.041312   53.797070  -33.394302   
2024-02-03   6.093106  106.182881    4.707807   24.604107  -16.126158   
2024-02-03  -9.063052  -20.553385  -31.627365  -28.418917   50.972916   
2024-02-03 -17.283341  -10.724938  -31.627365  -26.261679   88.716145   

                 PM10       PM2.5        SO2      